In [1]:
import pandas as pd
dt=pd.read_csv('info_preproc.csv')
dt.shape

(135144, 51)

In [2]:
# ””” 地 域 特 征 ”””
#省份
data_province_match = pd.read_excel('province_match.xls')
data_new = pd.merge(dt, data_province_match, left_on='city_num', right_on='市', how = 'left')#匹配城市所在的省，error的省为NAN
#删除重复行
data= data_new.drop_duplicates ( ['user_id'] )
df= data.reset_index(drop=True)#调整索引值
df ['省'].fillna('error',inplace=True) #空值用error填充


In [3]:
#””” 时 间 特 征 ”””
df['first_order_time'] = pd.to_datetime(df['first_order_time'])
#首单体验课时间
# 细 分 体 验 课 时 间 和 日 期
# 第 一 次 购 买 体 验 课 的 时 刻
df ['first_order_hour'] = df ['first_order_time' ].dt.hour

# 第 一 次 购 买 体 验 课 的 那 一 天， 31 号 处 理 为0号
df ['first_order_day'] = df ['first_order_time'].dt.day
df ['first_order_day'] = df ['first_order_day'].apply (lambda x : 0 if x==31 else x )

# 第 一 次 购 买 体 验 课 的 那 个 星 期 几
df ['first_order_week'] = df.first_order_time.apply(lambda x : x.weekday( ) + 1)

# 第 一 次 购 买 体 验 课 的 的 那 个 月 份
df [ 'first_order_month' ]= df ['first_order_time'].dt.month

# 第 一 次 购 买 体 验 课 的 的 那 一 年
df [ 'first_order_year' ]= df ['first_order_time'].dt.year


#第一次体验课订单的那个季度
def get_quarter(x):
    y=0
    if (1<=x<=3):
        y=1
    elif(4<=x<=6):
        y=2
    elif(7<=x<=9):
        y=3
    else:
        y=4
    return y
df ['first_order_Q'] = df.first_order_month.apply (lambda x : get_quarter ( x ) )

# 第 一 次 体 验 课 订 单 是 否 在 周 末
df [ 'first_order_is_weekend' ] = df.first_order_week.apply(lambda week: 1 if week>5 else 0)

import chinese_calendar
#第 一 次 体 验 课 订 单 是 否 在 中 国 节 假 日 期 间
df ['first_order_is_holiday'] = df.first_order_time.apply (lambda dt: 1 if chinese_calendar .is_holiday( dt ) else 0)
 
#第 一 次 体 验 课 订 单 是 否 在 工 作 日 期 间
df ['first_order_is_workday'] = df.first_order_time.apply (lambda dt: 1 if chinese_calendar.is_workday( dt ) else 0)

# 首单时间到最大日期的间隔天数，以最大的first_order_time 为表数据截止日期
max_datetime = max(df.first_order_time)
df['first_order_duration'] = pd.to_numeric(df.first_order_time.apply(lambda dt:(max_datetime-dt).days))
df['first_order_duration'].dtypes
#df.to_csv ( '1.csv',index=False,header=True )

dtype('int64')

In [4]:
#””” 订 单 特 征 ”””
# 第 一 次 体 验 课 订 单 是 否 免 费
df['first_order_is_free']=df.first_order_price.apply(lambda price:1 if price ==0 else 0 )#正常情况下是否免费对用户影响很大

# 第 一 次 体 验 课 订 单 是 否 小 于 10 元
#根据价格数量种类，十元以下属于交便宜的，十元以上直接到90多，属于贵的
df ['first_order_is_under_ten'] = df.first_order_price.apply(lambda price:1 if price <10 else 0 )

# 体 验 课 价 格 分 段
list_bins=[0,0.01,1,10,100,2100]
list_labels=['0-0.01','0.01-1','1-10','10-100','100+']
df['first_order_price_stage' ] = pd.cut( df ['first_order_price'],bins=list_bins, labels=list_labels, include_lowest=True)#包含左端点

In [5]:
# ””” 商 业 指 标 ”””
#省内用户转化率（用于可视化）
means = df.groupby('省')['result'].mean()
df['province_CVR'] = df['省'].map(means)


In [6]:
#””” 年 龄 特 征 ”””
list_bins=[0, 3.0, 6.0,12.0 ,18.0 ,2021]
list_labels=['0-3','3-6','6-12','12-18','18+']
df['age_stage' ] = pd.cut( df ['age_year'],bins=list_bins, labels=list_labels, include_lowest=True)#年龄分段，包含左端点

In [7]:
# ””” 行 为 特 征： 用 户 活 跃 度+统 计 指 标 ”””

# 平 均 Average 日 活 跃 度 = 登 录 天 数/体 验 天 数
df ['ADAU'] = df.login_day / df.first_order_duration
df ['ADAU'].fillna (1e-8, inplace=True )

# 平 均 Average 月 活 跃 度 = 登 录 天 数/体 验 月 数 （最 大 年 月 − 体 验 课 开 始）
df ['AMAU'] = df['ADAU'].apply(lambda x : x/30 if x!=0 else 10e-9)
# 平 均 Average 周 活 跃 度 = 登 录 天 数/体 验 周 数
df ['AWAU'] = df['ADAU'].apply(lambda x : x/7 if x!=0 else 10e-9)

# 登 陆 时 间 间 隔 相 较 于 均 值 的 占 比 和 差 值
df ['login_diff_time_mean_ratio'] = df.login_diff_time / df.login_diff_time.mean( )
df ['login_diff_time_diff_mean'] = df.login_diff_time - df.login_diff_time.mean( )

# 弹 窗 广 告 点 击 相 对 于 平 均 值 的 占 比 和 差 值
df ['ads_click_mean_ratio'] = df.click_dialog / df.click_dialog.mean( )
df ['ads_click_diff_mean'] = df.click_dialog-df.click_dialog.mean( )

# 分 享 数 share 相 对 于 平 均 值 的 占 比 和 差 值
df ['share_mean_ratio'] = df.share / df.share.mean( )
df ['share_diff_mean'] = df.share-df.share.mean( )


In [8]:
#””” 行 为 特 征： 用 户 学 习 情 况 ””” 
import math
# 学 习 完 成 率 = 完 成 课 程 节 数/学 习 课 程 节 数
df ['finish_learn_ratio'] = df.apply (lambda x : 100*x ['finish_num']/ x ['learn_num'] if x ['learn_num']>0 else 0 , axis =1)

# 登 陆 频 率 = 登 陆 天 数 / 总 天 数
df['login_day_ratio'] = df.login_day / df.first_order_duration
df.login_day_ratio.fillna(0, inplace=True )

# 平 均 每 天 在 线 时 长 = 登 录 时 长/登 录 天 数
df ['daily_study_time'] = df.apply (lambda x: x[ 'login_time' ] / x ['login_day'] if x ['login_day']>0 else 0 , axis =1)
# 大 于24 的 替 代 为 log ( x )−1
#df ['daily_study_time' ] = df.daily_study_time.apply(lambda x : math .log ( x )-1 if x>24 else x )

#是 否 订 阅 学 习 类 公 众 号
df['is_subscribe']=df['chinese_subscribe_num'].apply(lambda x:1 if x==1 else 0)
df['is_subscribe']=df['math_subscribe_num'].apply(lambda x:1 if x==1 else 0)


In [9]:
# ””” 统 计 特 征： 频 数””” 
# 设 备
# 设备类型，只有两种， 转 化 成 类 别 编 码 和 频 数 编 码，可能是 mobile + PC
df ['platform_num_cnt'] = df ['platform_num'].map(df.platform_num.value_counts( ))
df ['platform_num'] = df ['platform_num'].apply (lambda x : str( x ) )
# 手机型号
df['model_num_cnt'] = df ['model_num'].map(df.model_num .value_counts ( ))
# 城 市
df ['city_num_cnt'] = df ['city_num'].map( df.city_num.value_counts ( ))
df.city_num_cnt.fillna(0 , inplace=True )
# 省 份
df ['province_cnt'] = df ['省'] .map( df['省'].value_counts ( ))
df . province_cnt .fillna (0 ,inplace=True )
# 年 龄
df ['age_stage_cnt' ] = df [ 'age_stage'].map( df.age_stage.value_counts ( ) )
# 首 单 体 验 课 价 格
df ['first_order_price_stage_cnt' ] = df ['first_order_price_stage'].map( df.first_order_price_stage .value_counts ( ) )
#第一次体验课订单的季度
df ['first_order_Q_cnt'] = df ['first_order_Q']. map( df.first_order_Q.value_counts ( ) )

In [10]:
df

,user_id,first_order_time,first_order_price,age_month,city_num,platform_num,model_num,app_num,main_home,main_home2,...,login_day_ratio,daily_study_time,is_subscribe,platform_num_cnt,model_num_cnt,city_num_cnt,province_cnt,age_stage_cnt,first_order_price_stage_cnt,first_order_Q_cnt
0,2000001555945280,2018-12-23 11:44:00,0.00,32,广州,9.2969,11.2707,1,0.0,0.0,...,0.049645,0.142857,0,104887,554,3165,11406,11385.0,111112,49915
1,2000001556645228,2019-01-11 09:46:00,0.00,127,徐州,9.2969,4.9689,1,17.0,13.0,...,0.032787,0.750000,1,104887,300,732,4603,45419.0,111112,64155
2,2000001558047804,2018-12-26 11:04:00,0.00,92,error,9.2969,6.6392,1,5.0,13.0,...,0.007246,3.000000,0,104887,555,28422,29135,45419.0,111112,49915
3,2000001558146467,2018-12-31 08:47:00,0.00,83,error,9.2969,12.2222,1,72.0,43.0,...,0.045113,4.000000,0,104887,37,28422,29135,45419.0,111112,49915
4,2000001558146878,2019-01-28 01:53:00,0.00,47,error,13.557,10.3925,1,36.0,28.0,...,0.038095,9.750000,1,30257,1409,28422,29135,77133.0,111112,64155
5,2000001558147371,2019-04-25 22:32:00,0.00,41,珠海,9.2969,15.3846,1,41.0,104.0,...,0.222222,17.000000,0,104887,252,185,11406,77133.0,111112,21074
6,2000001559045233,2018-12-02 14:23:00,1.00,35,error,9.2969,6.7901,1,3.0,7.0,...,0.006173,8.000000,0,104887,132,28422,29135,11385.0,7066,49915
7,2000001559245920,2019-02-20 12:35:00,0.00,55,苏州,9.2969,18.7674,1,34.0,13.0,...,0.048780,9.500000,0,104887,483,828,4603,77133.0,111112,64155
8,2000001559246037,2019-01-13 08:53:00,0.00,55,重庆,13.557,10.4680,1,31.0,20.0,...,0.033333,1.500000,0,30257,531,12388,12388,77133.0,111112,64155
9,2000001559247825,2019-02-12 22:35:00,9.00,87,error,9.2969,6.4386,1,88.0,42.0,...,0.044444,7.750000,0,104887,128,28422,29135,45419.0,16880,64155


In [11]:
df.shape

(135144, 88)

In [12]:
df.to_csv ( 'feature_engineering.csv',index=False,header=True )